In [ ]:
# # 정답데이터 
# import os
# from PIL import Image

# # Define base paths
# base_dir = r"C:\Users\epdgn\Downloads\catvton\VITON-HD\test"
# output_dir = os.path.join(base_dir, "validation")

# # Ensure the output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Define subdirectories
# image_dir = os.path.join(base_dir, "image")
# densepose_dir = os.path.join(base_dir, "image-densepose")
# mask_dir = os.path.join(base_dir, "agnostic-mask")

# # Get all image filenames in the original image directory
# image_filenames = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]

# for file_name in image_filenames:
#     base_file_name = os.path.splitext(file_name)[0]  # Remove extension

#     # Define paths for each image type
#     original_image_path = os.path.join(image_dir, file_name)
#     densepose_image_path = os.path.join(densepose_dir, file_name)
#     mask_image_path = os.path.join(mask_dir, f"{base_file_name}_mask.png")

#     try:
#         # Open the images
#         original_image = Image.open(original_image_path)
#         densepose_image = Image.open(densepose_image_path)
#         mask_image = Image.open(mask_image_path)

#         # Create a new blank image large enough to contain all three images side-by-side
#         width, height = original_image.size
#         combined_width = width * 3
#         combined_height = height
#         combined_image = Image.new("RGB", (combined_width, combined_height))

#         # Paste the images into the combined image
#         combined_image.paste(original_image, (0, 0))
#         combined_image.paste(densepose_image, (width, 0))
#         combined_image.paste(mask_image.convert("RGB"), (width * 2, 0))  # Convert mask to RGB

#         # Save the combined image in the validation folder with the file name
#         output_path = os.path.join(output_dir, f"{base_file_name}_combined.jpg")
#         combined_image.save(output_path)

#         print(f"Combined image saved at: {output_path}")
#     except FileNotFoundError as e:
#         print(f"File not found for {file_name}: {e}")
#     except Exception as e:
#         print(f"An error occurred with {file_name}: {e}")


Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00006_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00008_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00013_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00017_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00034_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00035_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00055_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00057_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation\00064_00_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvt

In [4]:
from PIL import Image
import os
import numpy as np
import cv2
from model.cloth_masker import AutoMasker

# Initialize AutoMasker
automasker = AutoMasker()

# Define base paths
base_dir = r"C:\Users\epdgn\Downloads\catvton_test\trenbe\test"
output_dir = os.path.join(base_dir, "validation")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define subdirectories
original_dir = os.path.join(base_dir, "images")
cloth_dir = os.path.join(base_dir, "cloth")
prediction_dir = r"C:\Users\epdgn\Downloads\catvton_test\output\trenbe-512\result"
densepose_dir = os.path.join(base_dir, "images")
schp_lip_dir = os.path.join(base_dir, "images")
schp_atr_dir = os.path.join(base_dir, "images")
mask_dir = os.path.join(base_dir, "agnostic_masks")

# Standard size for all images
resize_width = 512
resize_height = 512

# Get all image filenames in the prediction directory
image_filenames = [f for f in os.listdir(prediction_dir) if f.endswith(".jpg")]

for file_name in image_filenames:
    base_file_name = os.path.splitext(file_name)[0]  # Remove extension

    # Define paths for each image type
    prediction_image_path = os.path.join(prediction_dir, file_name)
    densepose_image_path = os.path.join(densepose_dir, file_name)
    schp_lip_atr_path = os.path.join(schp_atr_dir, file_name)
    mask_path = os.path.join(mask_dir, f"{base_file_name}.png")
    original_image_path = os.path.join(original_dir, file_name)
    cloth_image_path = os.path.join(cloth_dir, file_name)

    # Open and resize the images
    prediction_image = Image.open(prediction_image_path).resize((resize_width, resize_height), Image.LANCZOS)
    original_image = Image.open(original_image_path).resize((resize_width, resize_height), Image.LANCZOS)
    cloth_image = Image.open(cloth_image_path).resize((resize_width, resize_height), Image.LANCZOS)

    # Use AutoMasker to process the images
    processed_images = automasker.preprocess_image(prediction_image_path)
    densepose_array = np.array(processed_images['densepose'])
    densepose_array = cv2.normalize(densepose_array, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    densepose_color = cv2.applyColorMap(densepose_array, cv2.COLORMAP_JET)
    densepose_image = Image.fromarray(cv2.cvtColor(densepose_color, cv2.COLOR_BGR2RGB)).resize((resize_width, resize_height), Image.LANCZOS)
    schp_lip_image = processed_images['schp_lip'].resize((resize_width, resize_height), Image.LANCZOS)
    schp_atr_image = processed_images['schp_atr'].resize((resize_width, resize_height), Image.LANCZOS)
    mask_image = Image.open(mask_path).resize((resize_width, resize_height), Image.LANCZOS)

    # Create a new blank image for 2 rows layout (top 3 images, bottom 4 images)
    combined_width = resize_width * 4  # Maximum width for 4 images in a row
    combined_height = resize_height * 2  # Maximum height for 2 rows
    combined_image = Image.new("RGB", (combined_width, combined_height))

    # Paste the images into the combined image
    # First row (original, cloth, prediction)
    combined_image.paste(original_image, (0, 0))
    combined_image.paste(cloth_image, (resize_width, 0))
    combined_image.paste(prediction_image, (resize_width * 2, 0))

    # Second row (densepose, schp_lip, schp_atr, mask)
    combined_image.paste(densepose_image, (0, resize_height))
    combined_image.paste(schp_lip_image, (resize_width, resize_height))
    combined_image.paste(schp_atr_image, (resize_width * 2, resize_height))
    combined_image.paste(mask_image, (resize_width * 3, resize_height))

    # Save the combined image in the validation folder with the file name
    output_path = os.path.join(output_dir, f"{base_file_name}_combined.jpg")
    combined_image.save(output_path)

    print(f"Combined image saved at: {output_path}")


Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\100_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\101_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\102_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\103_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\105_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\109_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\110_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\111_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation\112_combined.jpg
Combined image saved at: C:\Users\epdgn\Downloads\catvton_test\trenbe\tes

In [25]:
### 정답 데이터와 예측데이터 시각화 합치기
import os
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 경로 설정
ground_truth_dir = r"C:\Users\epdgn\Downloads\catvton\VITON-HD\test\validation"
prediction_dir = r"C:\Users\epdgn\Downloads\catvton_test\trenbe\test\validation"
output_dir = r"C:\Users\epdgn\Downloads\validation"

# 출력 디렉토리 생성 (이 경우 이미 지정된 경로에 저장할 예정)
os.makedirs(output_dir, exist_ok=True)

# 이미지 파일 목록 가져오기
ground_truth_files = [f for f in os.listdir(ground_truth_dir) if f.endswith(".jpg")]
prediction_files = [f for f in os.listdir(prediction_dir) if f.endswith(".jpg")]

# 이미지 처리 함수 (라벨 크기 조정)
def add_label(image, label):
    """
    이미지에 텍스트 레이블을 추가하는 함수
    """
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()  
    width, height = image.size
    
    label_font_size = 100  
    font = ImageFont.truetype("arial.ttf", label_font_size)
    
    label_position = (width // 2 - len(label) * 10, 10)  
    draw.text(label_position, label, font=font, fill="white")
    return image

# 두 폴더의 이미지들을 하나씩 결합
for ground_truth_file in ground_truth_files:
    if ground_truth_file in prediction_files:  
        ground_truth_path = os.path.join(ground_truth_dir, ground_truth_file)
        prediction_path = os.path.join(prediction_dir, ground_truth_file)

        ground_truth_image = Image.open(ground_truth_path)
        prediction_image = Image.open(prediction_path)

        ground_truth_image = add_label(ground_truth_image, "Ground Truth")
        prediction_image = add_label(prediction_image, "Prediction")

        gap = 50  
        
        total_height = ground_truth_image.height + prediction_image.height + gap
        combined_image = Image.new("RGB", (max(ground_truth_image.width, prediction_image.width), total_height))

        combined_image.paste(ground_truth_image, (0, 0))
        combined_image.paste(prediction_image, (0, ground_truth_image.height + gap))

        output_path = os.path.join(output_dir, f"{ground_truth_file}")
        combined_image.save(output_path)
        print(f"Saved combined image: {output_path}")


Saved combined image: C:\Users\epdgn\Downloads\validation\00006_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00008_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00013_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00017_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00034_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00035_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00055_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00057_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00064_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00067_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00069_00_combined.jpg
Saved combined image: C:\Users\epdgn\Downloads\validation\00071_00_combined.jpg
Saved combined image: C:\Users\epdgn\Dow